In [ ]:
import pandas as pd
from datetime import datetime
import pywhatkit as wpp

In [ ]:
def clean_asig(df):
    # Solo Colunnas necesarias. Filtro solo Cadereyta
    
    df = df[['MATRICULA','NOMBRE', 'CE', 'MATERIA']]
    
    # Matriculas en MAYUS
    df['MATRICULA'] = df['MATRICULA'].str.upper()
    
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    df = df[df['CE'].str.contains('|'.join(cade), case = False, na = False)]
    return df

In [ ]:
def colab():
    print('Grupos Colaborativos')
    grupo = str(input('Grupo ?'))
    # link = str(input('Link ?'))
    try: 
        asig = pd.read_csv(path + "Asignacion.csv", index_col= 0)
    except FileNotFoundError or FileExistsError:
        mes = datetime.now().month
        asig = extras = pd.read_excel(path + main + "Román - M{} Seguimiento.xlsx".format(mes-1), sheet_name = "Asignación")
        clean_asig(asig).to_csv(path + 'Asignacion.csv')
        print('Asignacion csv Created')
        asig = pd.read_csv(path + "Asignacion.csv", index_col= 0)
    if grupo:
        asig = asig[asig['MATERIA'] == grupo]
    asig_merged = pd.merge(asig, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return asig_merged 

In [ ]:
def test():
    path = 'Data/'
    df = pd.read_csv(path + 'Test.csv')
    df_merge = pd.merge(df, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return df_merge

Codigo

In [ ]:
path = 'Data/'
main = 'Main/'

try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")

In [ ]:
mencion_stat = ''
link = ''
path2 = 'infodata/'
imagen = 'img/Cerro.jpeg'

#Inicializar la infodata
try:
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2+ 'matriculas_yacontactadas.csv') 
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_error = pd.DataFrame()
    matric_yacontactadas = pd.DataFrame()

Opciones

In [ ]:
option = 0
while option not in (1, 2, 3):
    option = int(input('1.- Colab \n2.- Test Aprendizaje \n3.- No inicios'))
    if option == 1:
        link = str(input('Link? '))
        df = colab()
    elif option == 2:
        link = 'https://forms.office.com/r/xjAa2rPUhf'
        df = test()
    elif option == 3:
        print('')

Ciclo de mensajes

In [ ]:
for index, row in df.iterrows():
    alumno = row['NOMBRE']
    matricula = row['MATRICULA']
    telefono = row['Teléfono']
    
    if option == 1:
        materia = row['MATERIA']
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ tu Tutor Presencial del CNCI, me comunico contigo para invitarte al grupo colaborativo de la materia: *'+ materia + '* con la finalidad de que podamos entablar conversación por ahí sobre los temas relacionados y además tener apoyo entre los compañeros. \n\nSi gustas a continuación te dejo el enlace: ' + link
    elif option == 2:
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ tu Tutor Presencial del CNCI, me comunico contigo para invitarte a contestar el Test de Aprendizaje que tienes pendiente'
    
    if not pd.isna(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not pd.isna(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))       
    

    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        print('Enviado a: {} {}'.format(matricula, alumno))
        #wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)
        #print(telefono, matricula, alumno)